# Extract Frames from Video

In [ ]:
#
##
import torchvision
#
## define video to extract frames from
var_path_video = "../data/video/act_49_41.mp4"
var_name = var_path_video.split("/")[-1].split(".")[0]
#
## define frames to extract
var_frame_index = list(range(0, 90, 1))
print(var_name, var_frame_index)

In [ ]:
#
## read video
data_frame, _, var_info = torchvision.io.read_video(var_path_video)
data_frame.shape

In [ ]:
#
##
for var_i in var_frame_index:
    #
    data_image = data_frame[var_i].permute(2, 0, 1)
    #
    torchvision.io.write_jpeg(data_image, var_name + "_" + str(var_i) + ".jpg", 100)


# Visualize WiFi CSI

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as scio
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer

In [ ]:
#
##
var_path_mat = "../data/csi/mat/act_49_41.mat"
data_mat = scio.loadmat(var_path_mat)

In [ ]:
#
##
var_length = data_mat["trace"].shape[0]
#
## 
data_csi_amp = []
for var_t in range(var_length):
    data_csi_amp.append(abs(data_mat["trace"][var_t][0][0][0][-1]))
#
data_csi_amp = np.array(data_csi_amp, dtype = np.float32)
#
print(data_csi_amp.shape)

In [ ]:
#
##
data_csi_show = abs(data_csi_amp.reshape(data_csi_amp.shape[0], -1))
print(data_csi_show.shape)
#
##
data_csi_show = np.transpose(data_csi_show)
var_x_encoder = MinMaxScaler() 
data_csi_show = var_x_encoder.fit_transform(data_csi_show)
data_csi_show = np.transpose(data_csi_show)
#
##
var_f, _ = plt.subplots(1, 1)
plt.axis("off")
var_image = plt.imshow(data_csi_show.T, interpolation = "none", aspect = "auto", cmap = "viridis")

In [ ]:
var_name = var_path_mat.split("/")[-1].split(".")[0]
var_f.savefig(var_name + ".png", bbox_inches = "tight", pad_inches = 0, dpi = 600)